# Basic Steps

In [1]:
import os
import re
import nltk
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, SimpleRNN, TextVectorization, Dense, LSTM, Bidirectional
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
#COMMON USED CONSTATNS:
FILE_NAME = r'dataset/training.1600000.processed.noemoticon.csv'
COLUMNS = ['target','ids', 'date', 'flag', 'user', 'text']
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


1. load the dataset and display number of samples for each class label (0=neg, 
1=neutral, 4=pos)

In [ ]:
df = pd.read_csv(FILE_NAME, encoding='latin', header=None)
df.columns = COLUMNS
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

# Preprocessing

2. clean it to remove stop-words and punctuation

In [ ]:
def remove_punctations_and_stopwords(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = text.lower()
  text = [w for w in text.split() if w not in STOP_WORDS]
  return " ".join(text)
  
df['text'] = df['text'].apply(remove_punctations_and_stopwords)

In [2]:
Dir_Path = r'weights/'

def save_data(key, value):
  pickle.dump(value, open(os.path.join(Dir_Path, (key + '.pkl')), 'wb'))  
def load_data(key):
  return (pickle.load(open(os.path.join(Dir_Path, (key + '.pkl')), 'rb')))

* Transforming target values (0,4) to (0,1)

In [ ]:
def transform(x):
  if x == 4:
    return 1
  return x

df['target'] = df['target'].apply(transform) 

3. split dataset in train and test with 30-70 ratio, ensure stratified split

In [ ]:
X = df['text']
Y = df['target'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state=42, stratify=Y)

* converting inputs with vectors and generating vocabulary from this data

In [ ]:
text_vectorization = TextVectorization(max_tokens=10000, 
                                       output_sequence_length=140,
                                       output_mode="int")
text_vectorization.adapt(X)
X_train = text_vectorization(X_train)
X_test = text_vectorization(X_test)

save_data("X-train", X_train)
save_data("Y-train", Y_train)
save_data("X-test", X_test)
save_data("Y-test", Y_test)
save_data("TextVectorize", text_vectorization.get_vocabulary())

In [3]:
X_train = load_data("X-train")
Y_train = load_data("Y-train")

X_test = load_data("X-test")
Y_test = load_data("Y-test")

vocabulary = load_data("TextVectorize")

4. train following models while validating each of these on 20% of the training 
data, 

# RNN

* *1. RNN with an embedding layer and two hidden layers (64, 32) to learn 
word embedding from the data, keeping max feature size = 10000 and 
max length of each tweet 140 words*

In [ ]:
model = Sequential()
model.add(Embedding(10000, 140))
model.add(SimpleRNN(32))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-1(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-1.h5"))

Epoch 1/10
2987/2987 [==============================] - 447s 149ms/step - loss: 0.6614 - accuracy: 0.5964 - val_loss: 0.6117 - val_accuracy: 0.6800
Epoch 2/10
2987/2987 [==============================] - 444s 148ms/step - loss: 0.6240 - accuracy: 0.6619 - val_loss: 0.6539 - val_accuracy: 0.6147
Epoch 3/10
2987/2987 [==============================] - 394s 132ms/step - loss: 0.6205 - accuracy: 0.6666 - val_loss: 0.5959 - val_accuracy: 0.6883
Epoch 4/10
2987/2987 [==============================] - 394s 132ms/step - loss: 0.5926 - accuracy: 0.7008 - val_loss: 0.5649 - val_accuracy: 0.7324
Epoch 5/10
2987/2987 [==============================] - 389s 130ms/step - loss: 0.6107 - accuracy: 0.6806 - val_loss: 0.5979 - val_accuracy: 0.7107
Epoch 6/10
2987/2987 [==============================] - 389s 130ms/step - loss: 0.5653 - accuracy: 0.7202 - val_loss: 0.5343 - val_accuracy: 0.7590
Epoch 7/10
2987/2987 [==============================] - 398s 133ms/step - loss: 0.5419 - accuracy: 0.7417 - val_

* *2. Check the impact of adding an additional layer of 128 in first task*

In [ ]:
model = Sequential()
model.add(Embedding(10000, 140))
model.add(SimpleRNN(32))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-2(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-2.h5"))

Epoch 1/10
2987/2987 [==============================] - 430s 143ms/step - loss: 0.5695 - accuracy: 0.7107 - val_loss: 0.5089 - val_accuracy: 0.7674
Epoch 2/10
2987/2987 [==============================] - 397s 133ms/step - loss: 0.5687 - accuracy: 0.7137 - val_loss: 0.5784 - val_accuracy: 0.7165
Epoch 3/10
2987/2987 [==============================] - 404s 135ms/step - loss: 0.5485 - accuracy: 0.7460 - val_loss: 0.5740 - val_accuracy: 0.7288
Epoch 4/10
2987/2987 [==============================] - 395s 132ms/step - loss: 0.5175 - accuracy: 0.7623 - val_loss: 0.5359 - val_accuracy: 0.7559
Epoch 5/10
2987/2987 [==============================] - 397s 133ms/step - loss: 0.5129 - accuracy: 0.7616 - val_loss: 0.5108 - val_accuracy: 0.7771
Epoch 6/10
2987/2987 [==============================] - 397s 133ms/step - loss: 0.4780 - accuracy: 0.7804 - val_loss: 0.4683 - val_accuracy: 0.7810
Epoch 7/10
2987/2987 [==============================] - 420s 140ms/step - loss: 0.4633 - accuracy: 0.7860 - val_

* *3. Repeat 1 and 2, this time use GloVe with 100-dim instead of learning 
word-embedding through embedding layer*

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Datasets/glove.6B/glove.6B.100d.txt"
embeddings_index = {}
embedding_dim = 100

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

* For Task-1:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-3(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-3.h5"))

Epoch 1/10
2987/2987 [==============================] - 230s 76ms/step - loss: 0.6446 - accuracy: 0.6306 - val_loss: 0.6141 - val_accuracy: 0.6814
Epoch 2/10
2987/2987 [==============================] - 227s 76ms/step - loss: 0.6122 - accuracy: 0.6802 - val_loss: 0.6209 - val_accuracy: 0.6810
Epoch 3/10
2987/2987 [==============================] - 229s 77ms/step - loss: 0.5929 - accuracy: 0.6964 - val_loss: 0.5972 - val_accuracy: 0.6994
Epoch 4/10
2987/2987 [==============================] - 229s 77ms/step - loss: 0.5841 - accuracy: 0.7050 - val_loss: 0.5858 - val_accuracy: 0.6963
Epoch 5/10
2987/2987 [==============================] - 226s 76ms/step - loss: 0.5697 - accuracy: 0.7128 - val_loss: 0.5563 - val_accuracy: 0.7230
Epoch 6/10
2987/2987 [==============================] - 270s 90ms/step - loss: 0.5635 - accuracy: 0.7135 - val_loss: 0.5626 - val_accuracy: 0.7162
Epoch 7/10
2987/2987 [==============================] - 280s 94ms/step - loss: 0.5751 - accuracy: 0.7042 - val_loss: 0

* For Task-2:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-4(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-4.h5"))

Epoch 1/10
2987/2987 [==============================] - 240s 80ms/step - loss: 0.6630 - accuracy: 0.6040 - val_loss: 0.6694 - val_accuracy: 0.6054
Epoch 2/10
2987/2987 [==============================] - 237s 79ms/step - loss: 0.6179 - accuracy: 0.6679 - val_loss: 0.5869 - val_accuracy: 0.6949
Epoch 3/10
2987/2987 [==============================] - 237s 79ms/step - loss: 0.5977 - accuracy: 0.6829 - val_loss: 0.5814 - val_accuracy: 0.7018
Epoch 4/10
2987/2987 [==============================] - 239s 80ms/step - loss: 0.5951 - accuracy: 0.6872 - val_loss: 0.5946 - val_accuracy: 0.6950
Epoch 5/10
2987/2987 [==============================] - 239s 80ms/step - loss: 0.6336 - accuracy: 0.6373 - val_loss: 0.6328 - val_accuracy: 0.6496
Epoch 6/10
2987/2987 [==============================] - 246s 82ms/step - loss: 0.5941 - accuracy: 0.6875 - val_loss: 0.5639 - val_accuracy: 0.7114
Epoch 7/10
2987/2987 [==============================] - 239s 80ms/step - loss: 0.6087 - accuracy: 0.6535 - val_loss: 0

* *4. Repeat 3, this time use GloVe-twitter instead of GloVe*

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Datasets/glove.twitter.27B.100d.txt"
embeddings_index = {}
embedding_dim = 100

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

* For Task-1:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-5(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-5.h5"))

Epoch 1/10
2987/2987 [==============================] - 247s 82ms/step - loss: 0.6415 - accuracy: 0.6308 - val_loss: 0.6237 - val_accuracy: 0.6566
Epoch 2/10
2987/2987 [==============================] - 249s 83ms/step - loss: 0.6002 - accuracy: 0.6940 - val_loss: 0.5732 - val_accuracy: 0.7136
Epoch 3/10
2987/2987 [==============================] - 243s 81ms/step - loss: 0.5641 - accuracy: 0.7162 - val_loss: 0.5409 - val_accuracy: 0.7207
Epoch 4/10
2987/2987 [==============================] - 249s 83ms/step - loss: 0.5526 - accuracy: 0.7227 - val_loss: 0.5512 - val_accuracy: 0.7209
Epoch 5/10
2987/2987 [==============================] - 271s 91ms/step - loss: 0.6116 - accuracy: 0.6496 - val_loss: 0.6004 - val_accuracy: 0.6860
Epoch 6/10
2987/2987 [==============================] - 252s 84ms/step - loss: 0.5546 - accuracy: 0.7190 - val_loss: 0.5536 - val_accuracy: 0.7290
Epoch 7/10
2987/2987 [==============================] - 256s 86ms/step - loss: 0.5260 - accuracy: 0.7402 - val_loss: 0

* For Task-2:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-6(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-6.h5"))

Epoch 1/10
2987/2987 [==============================] - 258s 86ms/step - loss: 0.6387 - accuracy: 0.6377 - val_loss: 0.5823 - val_accuracy: 0.7073
Epoch 2/10
2987/2987 [==============================] - 256s 86ms/step - loss: 0.5823 - accuracy: 0.7012 - val_loss: 0.5864 - val_accuracy: 0.6991
Epoch 3/10
2987/2987 [==============================] - 250s 84ms/step - loss: 0.6129 - accuracy: 0.6760 - val_loss: 0.6184 - val_accuracy: 0.6653
Epoch 4/10
2987/2987 [==============================] - 249s 83ms/step - loss: 0.5780 - accuracy: 0.7106 - val_loss: 0.5457 - val_accuracy: 0.7306
Epoch 5/10
2987/2987 [==============================] - 248s 83ms/step - loss: 0.5506 - accuracy: 0.7255 - val_loss: 0.5521 - val_accuracy: 0.7202
Epoch 6/10
2987/2987 [==============================] - 250s 84ms/step - loss: 0.5457 - accuracy: 0.7280 - val_loss: 0.5188 - val_accuracy: 0.7441
Epoch 7/10
2987/2987 [==============================] - 249s 83ms/step - loss: 0.5185 - accuracy: 0.7438 - val_loss: 0

* *5. Repeat 3, replace GloVe with FastText*

In [ ]:
path_to_fasttext_file = "/content/drive/MyDrive/Datasets/FastText(wiki-news-300d-1M).vec"
embeddings_index = {}
embedding_dim = 300

with open(path_to_fasttext_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

* For Task-1:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-7(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-7.h5"))

Epoch 1/10
2987/2987 [==============================] - 507s 169ms/step - loss: 0.6195 - accuracy: 0.6673 - val_loss: 0.6366 - val_accuracy: 0.6616
Epoch 2/10
2987/2987 [==============================] - 419s 140ms/step - loss: 0.5966 - accuracy: 0.6936 - val_loss: 0.6162 - val_accuracy: 0.6937
Epoch 3/10
2987/2987 [==============================] - 404s 135ms/step - loss: 0.6080 - accuracy: 0.6802 - val_loss: 0.6242 - val_accuracy: 0.6687
Epoch 4/10
2987/2987 [==============================] - 403s 135ms/step - loss: 0.6074 - accuracy: 0.6829 - val_loss: 0.5907 - val_accuracy: 0.6973
Epoch 5/10
2987/2987 [==============================] - 401s 134ms/step - loss: 0.6330 - accuracy: 0.6450 - val_loss: 0.6364 - val_accuracy: 0.6530
Epoch 6/10
2987/2987 [==============================] - 401s 134ms/step - loss: 0.6176 - accuracy: 0.6648 - val_loss: 0.6572 - val_accuracy: 0.6297
Epoch 7/10
2987/2987 [==============================] - 401s 134ms/step - loss: 0.6407 - accuracy: 0.6372 - val_

 * For Task-2:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(SimpleRNN(32))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-8(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-8.h5"))

Epoch 1/10
2987/2987 [==============================] - 350s 116ms/step - loss: 0.6484 - accuracy: 0.6228 - val_loss: 0.6131 - val_accuracy: 0.6787
Epoch 2/10
2987/2987 [==============================] - 345s 115ms/step - loss: 0.6131 - accuracy: 0.6775 - val_loss: 0.6037 - val_accuracy: 0.6885
Epoch 3/10
2987/2987 [==============================] - 349s 117ms/step - loss: 0.5856 - accuracy: 0.7070 - val_loss: 0.5868 - val_accuracy: 0.7134
Epoch 4/10
2987/2987 [==============================] - 342s 115ms/step - loss: 0.5684 - accuracy: 0.7191 - val_loss: 0.5752 - val_accuracy: 0.7208
Epoch 5/10
2987/2987 [==============================] - 343s 115ms/step - loss: 0.5607 - accuracy: 0.7224 - val_loss: 0.5347 - val_accuracy: 0.7398
Epoch 6/10
2987/2987 [==============================] - 340s 114ms/step - loss: 0.5476 - accuracy: 0.7314 - val_loss: 0.5263 - val_accuracy: 0.7463
Epoch 7/10
2987/2987 [==============================] - 339s 114ms/step - loss: 0.5323 - accuracy: 0.7407 - val_

# LSTM

* *6. Repeat 1-5, replace RNN with LSTM*



* Task-1 (with LSTM)


In [ ]:
model = Sequential()
model.add(Embedding(10000, 140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-9(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-9.h5"))

Epoch 1/10
2987/2987 [==============================] - 80s 23ms/step - loss: 0.4633 - accuracy: 0.7788 - val_loss: 0.4419 - val_accuracy: 0.7911
Epoch 2/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4341 - accuracy: 0.7969 - val_loss: 0.4367 - val_accuracy: 0.7954
Epoch 3/10
2987/2987 [==============================] - 62s 21ms/step - loss: 0.4232 - accuracy: 0.8033 - val_loss: 0.4344 - val_accuracy: 0.7966
Epoch 4/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.4149 - accuracy: 0.8086 - val_loss: 0.4372 - val_accuracy: 0.7967
Epoch 5/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4080 - accuracy: 0.8124 - val_loss: 0.4351 - val_accuracy: 0.7957
Epoch 6/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.4018 - accuracy: 0.8161 - val_loss: 0.4335 - val_accuracy: 0.7960
Epoch 7/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.3959 - accuracy: 0.8199 - val_loss: 0.4325 -

* Task-2

In [ ]:
model = Sequential()
model.add(Embedding(10000, 140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-10(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-10.h5"))

Epoch 1/10
2987/2987 [==============================] - 73s 23ms/step - loss: 0.4632 - accuracy: 0.7789 - val_loss: 0.4436 - val_accuracy: 0.7911
Epoch 2/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.4341 - accuracy: 0.7973 - val_loss: 0.4367 - val_accuracy: 0.7942
Epoch 3/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4231 - accuracy: 0.8038 - val_loss: 0.4327 - val_accuracy: 0.7965
Epoch 4/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.4151 - accuracy: 0.8088 - val_loss: 0.4313 - val_accuracy: 0.7980
Epoch 5/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4082 - accuracy: 0.8126 - val_loss: 0.4352 - val_accuracy: 0.7954
Epoch 6/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4016 - accuracy: 0.8166 - val_loss: 0.4336 - val_accuracy: 0.7964
Epoch 7/10
2987/2987 [==============================] - 62s 21ms/step - loss: 0.3955 - accuracy: 0.8204 - val_loss: 0.4351 -

* Task-3

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Datasets/glove.6B/glove.6B.100d.txt"
embeddings_index = {}
embedding_dim = 100

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

 Part-A

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-11(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-11.h5"))

Epoch 1/10
2987/2987 [==============================] - 58s 18ms/step - loss: 0.5187 - accuracy: 0.7417 - val_loss: 0.4930 - val_accuracy: 0.7603
Epoch 2/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4812 - accuracy: 0.7670 - val_loss: 0.4726 - val_accuracy: 0.7735
Epoch 3/10
2987/2987 [==============================] - 51s 17ms/step - loss: 0.4698 - accuracy: 0.7745 - val_loss: 0.4667 - val_accuracy: 0.7766
Epoch 4/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4630 - accuracy: 0.7789 - val_loss: 0.4626 - val_accuracy: 0.7793
Epoch 5/10
2987/2987 [==============================] - 51s 17ms/step - loss: 0.4584 - accuracy: 0.7816 - val_loss: 0.4717 - val_accuracy: 0.7743
Epoch 6/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4551 - accuracy: 0.7835 - val_loss: 0.4582 - val_accuracy: 0.7820
Epoch 7/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4524 - accuracy: 0.7852 - val_loss: 0.4617 -

Part-B

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-12(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-12.h5"))

Epoch 1/10
2987/2987 [==============================] - 57s 18ms/step - loss: 0.5179 - accuracy: 0.7422 - val_loss: 0.5041 - val_accuracy: 0.7511
Epoch 2/10
2987/2987 [==============================] - 54s 18ms/step - loss: 0.4811 - accuracy: 0.7669 - val_loss: 0.4738 - val_accuracy: 0.7720
Epoch 3/10
2987/2987 [==============================] - 52s 18ms/step - loss: 0.4696 - accuracy: 0.7742 - val_loss: 0.4736 - val_accuracy: 0.7724
Epoch 4/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4665 - accuracy: 0.7766 - val_loss: 0.4655 - val_accuracy: 0.7786
Epoch 5/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4585 - accuracy: 0.7814 - val_loss: 0.4605 - val_accuracy: 0.7808
Epoch 6/10
2987/2987 [==============================] - 51s 17ms/step - loss: 0.4548 - accuracy: 0.7836 - val_loss: 0.4590 - val_accuracy: 0.7820
Epoch 7/10
2987/2987 [==============================] - 51s 17ms/step - loss: 0.4532 - accuracy: 0.7844 - val_loss: 0.4585 -

* Task-4

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Datasets/glove.twitter.27B.100d.txt"
embeddings_index = {}
embedding_dim = 100

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Part-A

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-13(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-13.h5"))

Epoch 1/10
2987/2987 [==============================] - 57s 18ms/step - loss: 0.4889 - accuracy: 0.7610 - val_loss: 0.4667 - val_accuracy: 0.7764
Epoch 2/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4614 - accuracy: 0.7789 - val_loss: 0.4584 - val_accuracy: 0.7819
Epoch 3/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4523 - accuracy: 0.7848 - val_loss: 0.4543 - val_accuracy: 0.7832
Epoch 4/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4471 - accuracy: 0.7882 - val_loss: 0.4485 - val_accuracy: 0.7866
Epoch 5/10
2987/2987 [==============================] - 52s 17ms/step - loss: 0.4432 - accuracy: 0.7907 - val_loss: 0.4463 - val_accuracy: 0.7893
Epoch 6/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4402 - accuracy: 0.7926 - val_loss: 0.4459 - val_accuracy: 0.7885
Epoch 7/10
2987/2987 [==============================] - 51s 17ms/step - loss: 0.4379 - accuracy: 0.7939 - val_loss: 0.4456 -

Part-B

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-14(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-14.h5"))

Epoch 1/10
2987/2987 [==============================] - 58s 18ms/step - loss: 0.4901 - accuracy: 0.7600 - val_loss: 0.4688 - val_accuracy: 0.7744
Epoch 2/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4622 - accuracy: 0.7787 - val_loss: 0.4562 - val_accuracy: 0.7816
Epoch 3/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4531 - accuracy: 0.7844 - val_loss: 0.4514 - val_accuracy: 0.7848
Epoch 4/10
2987/2987 [==============================] - 55s 18ms/step - loss: 0.4474 - accuracy: 0.7877 - val_loss: 0.4486 - val_accuracy: 0.7866
Epoch 5/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4435 - accuracy: 0.7902 - val_loss: 0.4474 - val_accuracy: 0.7882
Epoch 6/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4406 - accuracy: 0.7919 - val_loss: 0.4438 - val_accuracy: 0.7901
Epoch 7/10
2987/2987 [==============================] - 53s 18ms/step - loss: 0.4383 - accuracy: 0.7933 - val_loss: 0.4499 -

* Task-5

In [ ]:
path_to_fasttext_file = "/content/drive/MyDrive/Datasets/FastText(wiki-news-300d-1M).vec"
embeddings_index = {}
embedding_dim = 300

with open(path_to_fasttext_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

word_index = dict(zip(vocabulary, range(len(vocabulary))))
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Part-A:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-15(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-15.h5"))

Epoch 1/10
2987/2987 [==============================] - 63s 20ms/step - loss: 0.4940 - accuracy: 0.7585 - val_loss: 0.4724 - val_accuracy: 0.7736
Epoch 2/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4635 - accuracy: 0.7780 - val_loss: 0.4558 - val_accuracy: 0.7834
Epoch 3/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4507 - accuracy: 0.7861 - val_loss: 0.4513 - val_accuracy: 0.7871
Epoch 4/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4437 - accuracy: 0.7906 - val_loss: 0.4448 - val_accuracy: 0.7902
Epoch 5/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4386 - accuracy: 0.7934 - val_loss: 0.4431 - val_accuracy: 0.7911
Epoch 6/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4347 - accuracy: 0.7959 - val_loss: 0.4425 - val_accuracy: 0.7912
Epoch 7/10
2987/2987 [==============================] - 59s 20ms/step - loss: 0.4315 - accuracy: 0.7977 - val_loss: 0.4376 -

Part-B:

In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_dim, input_length=140))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=300, validation_split=0.2)
model.save(os.path.join(Dir_Path, "models/model-16(arc).h5"))
model.save_weights(os.path.join(Dir_Path, "models/model-16.h5"))

Epoch 1/10
2987/2987 [==============================] - 68s 22ms/step - loss: 0.4958 - accuracy: 0.7577 - val_loss: 0.4889 - val_accuracy: 0.7616
Epoch 2/10
2987/2987 [==============================] - 62s 21ms/step - loss: 0.4621 - accuracy: 0.7793 - val_loss: 0.4554 - val_accuracy: 0.7842
Epoch 3/10
2987/2987 [==============================] - 61s 20ms/step - loss: 0.4506 - accuracy: 0.7860 - val_loss: 0.4488 - val_accuracy: 0.7876
Epoch 4/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4439 - accuracy: 0.7903 - val_loss: 0.4523 - val_accuracy: 0.7857
Epoch 5/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4385 - accuracy: 0.7933 - val_loss: 0.4419 - val_accuracy: 0.7917
Epoch 6/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4349 - accuracy: 0.7953 - val_loss: 0.4424 - val_accuracy: 0.7920
Epoch 7/10
2987/2987 [==============================] - 60s 20ms/step - loss: 0.4318 - accuracy: 0.7970 - val_loss: 0.4381 -

# Result Section

In [4]:
experiments = {'Experiments':['RNN with (64,32) Hidden Layers',
                              'RNN with (128, 64,32) Hidden Layers',
                              'RNN (64,32) with Glove-100dim',
                              'RNN (128, 64,32) with Glove-100dim',
                              'RNN (64,32) with GloveTwitter-100dim',
                              'RNN (128, 64,32) with GloveTwitter-100dim',
                              'RNN (64,32) with FastText-300dim',
                              'RNN (128, 64,32) with FastText-300dim',
                              'LSTM with (64,32) Hidden Layers',
                              'LSTM with (128, 64,32) Hidden Layers',
                              'LSTM (64,32) with Glove-100dim',
                              'LSTM (128, 64,32) with Glove-100dim',
                              'LSTM (64,32) with GloveTwitter-100dim',
                              'LSTM (128, 64,32) with GloveTwitter-100dim',
                              'LSTM (64,32) with FastText-300dim',
                              'LSTM (128, 64,32) with FastText-300dim'], 
               'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-Score': []}

In [5]:
def results(model_no, rows):
  model = tf.keras.models.load_model(os.path.join(Dir_Path, f'models/model-{model_no}(arc).h5'))
  model.load_weights(os.path.join(Dir_Path, f'models/model-{model_no}.h5'))
  Y_predict = (model.predict(X_test)).round()
  rows['Accuracy'].append(accuracy_score(Y_test, Y_predict))
  rows['Precision'].append(precision_score(Y_test, Y_predict))
  rows['Recall'].append(recall_score(Y_test, Y_predict))
  rows['F1-Score'].append(f1_score(Y_test, Y_predict))

for i in range(1, 17):
  results(i, experiments)

15000/15000 [==============================] - 94s 6ms/step


In [30]:
result_table = pd.DataFrame(experiments)
result_table.to_excel(os.path.join(Dir_Path, "Experiment Results.xlsx"))
result_table = result_table.round(decimals=2) 
result_table

,Experiments,Accuracy,Precision,Recall,F1-Score
0,"RNN with (64,32) Hidden Layers",0.77,0.74,0.84,0.79
1,"RNN with (128, 64,32) Hidden Layers",0.77,0.83,0.69,0.75
2,"RNN (64,32) with Glove-100dim",0.72,0.76,0.63,0.69
3,"RNN (128, 64,32) with Glove-100dim",0.68,0.75,0.55,0.64
4,"RNN (64,32) with GloveTwitter-100dim",0.56,0.57,0.48,0.52
5,"RNN (128, 64,32) with GloveTwitter-100dim",0.51,0.50,0.73,0.60
6,"RNN (64,32) with FastText-300dim",0.70,0.81,0.52,0.63
7,"RNN (128, 64,32) with FastText-300dim",0.75,0.80,0.66,0.72
8,"LSTM with (64,32) Hidden Layers",0.79,0.81,0.77,0.79
9,"LSTM with (128, 64,32) Hidden Layers",0.79,0.77,0.83,0.80


**Best Model:** I mean in my assignment, i have observed all of the models they performing very well just a little bit difference among them. However, but I noticed that when I train the LSTM-(128, 64, 32) with Hidden Layers it performs fantastics the training accuracy around 83% i got also when it performed on testing data it is also giving us around 78% which is good enough. 